In [13]:
from netCDF4 import Dataset
import numpy as np
import json
from netCDFfunc.utility import get_data_by_date, get_stat, get_data_A
import os

In [2]:
dsin = Dataset(f'/Volumes/T7/AVHRR_OI_SST/v2.1/1981/oisst-avhrr-v02r01.19810901.nc', 'r', format='NETCDF4')

In [3]:
dsout = Dataset(f'/Volumes/T7/new_data/test.ncdf', 'w', format='NETCDF4')

## copy data

In [4]:
# set attribute
for attr in dsin.ncattrs() :
    dsout.setncattr(attr, dsin.getncattr(attr))

In [5]:
# set dimension
for k, v in dsin.dimensions.items():
    dsout.createDimension(k, v.size)

In [6]:
# set variables
for name, variable in dsin.variables.items():
    
    new_variable = dsout.createVariable(name, variable.datatype, variable.dimensions, fill_value=-999) # name, datatype, dimension
    dsout[name][:] = dsin[name][:] # values
    
    for attr in variable.ncattrs():
        if attr == '_FillValue': continue
        new_variable.setncattr(attr, variable.getncattr(attr)) # variable attr

## write data

In [ ]:
start_date = (1981,9,1)
end_date = (2011,8,31)
result = get_data_by_date(get_data_A, 'sst', start_date, end_date)

  0%|          | 0/31 [00:00<?, ?it/s]

In [8]:
mean = get_stat(result, 'mean')

0it [00:00, ?it/s]

In [10]:
def create_new_variable(dsin, dsout, new_varable_name, dtype, dimension, fill_value, values, attributes):
    for attr in dsin.ncattrs() :
        dsout.setncattr(attr, dsin.getncattr(attr))

    for k, v in dsin.dimensions.items():
        dsout.createDimension(k, v.size)
    
    for name, variable in dsin.variables.items():
    
        existing_variable = dsout.createVariable(name, variable.datatype, variable.dimensions, fill_value=fill_value) # name, datatype, dimension
        dsout[name][:] = dsin[name][:] # values

        for attr in variable.ncattrs():
            if attr == '_FillValue': continue
            existing_variable.setncattr(attr, variable.getncattr(attr)) # variable attr
            
    new_variable = dsout.createVariable(new_varable_name, dtype, dimension, fill_value=fill_value)
    dsout[new_varable_name][:] = values
    
    for k,v in attributes.items():
        if attr == '_FillValue': continue
        new_variable.setncattr(k, v) # variable attr
    
    return dsout

In [11]:
def transfer_data(year, month, date, variable_type, variable_value): 
    '''
    variable_type = 'mean' or 'perc' or 'cnt'
    '''
    
    date = dt.date(year,month,day).strftime('%Y%m%d')
    
    if year < 2016 : 
        dsin = Dataset(f'/Volumes/T7/AVHRR_OI_SST/v2.1/{year}/oisst-avhrr-v02r01.{date}.nc', 'r', format='NETCDF4')
    else :
        if os.path.exists(f'/Volumes/T7/AVHRR_OI_SST/v2.1/{year}/{date}120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.0-fv02.1.nc'):
            dsin = Dataset(f'/Volumes/T7/AVHRR_OI_SST/v2.1/{year}/{date}120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.0-fv02.1.nc', 'r', format='NETCDF4')
        else :
            dsin = Dataset(f'/Volumes/T7/AVHRR_OI_SST/v2.1/{year}/{date}120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.1-fv02.1.nc', 'r', format='NETCDF4')
    
    if not os.path.exists(f'/Volumes/T7/new_data/AVHRR_OI_SST/{year}'):
        os.makedirs((f'/Volumes/T7/new_data/AVHRR_OI_SST/{year}'))
                           
    dsout = Dataset(f'/Volumes/T7/new_data/AVHRR_OI_SST/{year}/{date}120000-NCEI-L4_GHRSST-SSTblend-AVHRR_OI-GLOB-v02.1-fv02.1.nc', 'w', format='NETCDF4')

    if variable_type == 'mean' :
        variable_name = 'mean_sst'
        dtype = np.float32
        attribute = {'long_name' : 'mean sst for 30 years (1981/9/1~2011/8/31)',
                     'add_offset' : 0.0,
                     'scale_factor' : 0.01,
                     'valid_min' : -1200,
                     'valid_max' : 1200,
                     'units' : 'celcius'}
        
    if variable_type == 'perc' :
        variable_name = '90-percentile_sst'
        dtype = np.float32
        attribute = {'long_name' : '90-percentile of sst for 30years (1981/9/1~2011/8/31)',
                     'add_offset' : 0.0,
                     'scale_factor' : 0.01,
                     'valid_min' : -1200,
                     'valid_max' : 1200,
                     'units' : 'celcius'}
    
    if variable_type == 'cnt' :
        variable_name = 'cnt_of_value'
        dtype = np.int16
        attribute = {'long_name' : 'based values counted by pixel (30,0 = normal, other = anormal)',
                     'add_offset' : 0,
                     'scale_factor' : 1,
                     'valid_min' : 0,
                     'valid_max' : 30,
                     'units' : '-'}
        
              
    
    # common attribute
    dimensions = ('time', 'zlev', 'lat', 'lon')
    fill_value = -999
    values = variable_value[(month,date)]

    create_new_variable(dsin, dsout, variable_name, dtype, dimensions, fill_value, values, attribute)

    dsout.close()
    dsin.close()

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: NOAA/NCEI 1/4 Degree Daily Optimum Interpolation Sea Surface Temperature (OISST) Analysis, Version 2.1 - Final
    source: ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfinder_AVHRR, Navy_AVHRR
    id: oisst-avhrr-v02r01.19810901.nc
    naming_authority: gov.noaa.ncei
    summary: NOAAs 1/4-degree Daily Optimum Interpolation Sea Surface Temperature (OISST) (sometimes referred to as Reynolds SST, which however also refers to earlier products at different resolution), currently available as version v02r01, is created by interpolating and extrapolating SST observations from different sources, resulting in a smoothed complete field. The sources of data are satellite (AVHRR) and in situ platforms (i.e., ships and buoys), and the specific datasets employed may change over time. At the marginal ice zone, sea ice concentrations are used to generate proxy SSTs.  A preliminary version of this file is pro

In [12]:
dsout.close()
dsin.close()

In [10]:
dsbackup = dsout

In [11]:
dsbackup

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: NOAA/NCEI 1/4 Degree Daily Optimum Interpolation Sea Surface Temperature (OISST) Analysis, Version 2.1 - Final
    source: ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfinder_AVHRR, Navy_AVHRR
    id: oisst-avhrr-v02r01.19810901.nc
    naming_authority: gov.noaa.ncei
    summary: NOAAs 1/4-degree Daily Optimum Interpolation Sea Surface Temperature (OISST) (sometimes referred to as Reynolds SST, which however also refers to earlier products at different resolution), currently available as version v02r01, is created by interpolating and extrapolating SST observations from different sources, resulting in a smoothed complete field. The sources of data are satellite (AVHRR) and in situ platforms (i.e., ships and buoys), and the specific datasets employed may change over time. At the marginal ice zone, sea ice concentrations are used to generate proxy SSTs.  A preliminary version of this file is pro

In [17]:
dsout = Dataset(f'/Volumes/T7/new_data/AVHRR_OI_SST/oisst-avhrr-v02r01.19810901.nc', 'r', format='NETCDF4')

masked_array(
  data=[[[[        nan,         nan,         nan, ...,         nan,
                   nan,         nan],
          [        nan,         nan,         nan, ...,         nan,
                   nan,         nan],
          [        nan,         nan,         nan, ...,         nan,
                   nan,         nan],
          ...,
          [-0.01212333, -0.01212333, -0.01212333, ..., -0.01212333,
           -0.01212333, -0.01212333],
          [-0.01210333, -0.01210333, -0.01210333, ..., -0.01210667,
           -0.01210333, -0.01210333],
          [-0.01211   , -0.01211   , -0.01211   , ..., -0.01211   ,
           -0.01211   , -0.01211   ]]]],
  mask=False,
  fill_value=1e+20,
  dtype=float32)